In [1]:
# pip install scrapy
# pip install --upgrade zope2

from collections import Counter
import foursquare
import json
import matplotlib.pyplot as plt
import numpy as np
import operator
from operator import itemgetter
import pandas as pd
import pickle
import random
import requests
from scrapy import Selector
from scrapy.http import HtmlResponse
import seaborn as sns
from sqlalchemy import create_engine
import sqlite3
import textacy
import textacy.data
import time
import unicodedata

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
usda = pd.read_pickle('../../../../Accounts/Electo:Computad/\
MacBook Pro/Software/Github/Old_repos/DSI-SF-4-anotherguitar/projects/Capstone Stuff/usda.pkl')
food = pd.read_pickle('../../../../Accounts/Electo:Computad/\
MacBook Pro/Software/Github/Old_repos/DSI-SF-4-anotherguitar/projects/Capstone Stuff/explore_endpoint_df.pkl')
vegan = food[food.vegan_venue_check == 'vegan']

In [3]:
def word_counter(unique_desc):
    single_string = ' '.join(unique_desc)
    count_string = len(single_string.split())
    return count_string

In [4]:
# have to count the number of words in each list and add this as a new column to the dataframe
updated_food = food
updated_food['count_desc_list'] = food.venue_desc_list.map(lambda x: word_counter(x))

In [5]:
def item_desc_counter(phrase):
    if phrase == 'desc_not_available':
        return 0
    else:
        countered = len(phrase.split())
        return countered

In [6]:
updated_food['count_item_name'] = food.menu_item_name.map(lambda x: len(x.split()))
updated_food['count_item_desc'] = food.menu_item_desc.map(lambda x: item_desc_counter(x))

In [7]:
updated_vegan = updated_food[updated_food.vegan_venue_check == 'vegan']

In [8]:
usda = usda.drop('ComName', axis=1)
usda = usda.drop('FG_Cd', axis=1)

In [9]:
usda.head()

,Food_No,Food_Group,Description,Nutr_No,Nutrient,Amount_per_100,Units,Tagname,Not_Imputed
0,01001,Dairy and Egg Products,"Butter, salted",203,Protein,0.85,g,PROCNT,1
1,01001,Dairy and Egg Products,"Butter, salted",204,Total lipid (fat),81.11,g,FAT,1
2,01001,Dairy and Egg Products,"Butter, salted",205,"Carbohydrate, by difference",0.06,g,CHOCDF,0
3,01001,Dairy and Egg Products,"Butter, salted",207,Ash,2.11,g,ASH,1
4,01001,Dairy and Egg Products,"Butter, salted",208,Energy,717.00,kcal,ENERC_KCAL,0


In [10]:
dads = usda[['Food_Group','Description']]

dads.Food_Group.unique()

# Columns I may drop/rationale:
# Probably nix Baby Foods (just baby versions, no unique identifier, so doesn't have predictive value)
# Probably nix Restaurant Foods (just restaurant versions, with some brand names, so doesn't have predictive value)
# Probably nix Breakfast Cereals - primarily brand names, w/ some 'cream' - nothing else helpful
# American Indian/Alaska Native Foods because it has a lot of fish, and is confusing my categories (I think)
# I found 'cream' in Soups, Sauces, and Gravies, but pultry products come up a lot too - check what id's come
# out of dairy

# Notes:
# On the fence: maybe nix Fast Foods, because it tends to be less informative (also the meals, entrees one)
# On the fence: Spices & Herbs (has some unique ingredients, but also generic words)
# Fats and Oils includes Mayo, make a unique one for mayo - is spelled as mayo, or full mayoneise on unique menus
#          ? b/c i think it becomes egg then, also fish oil, soy,
#       margarine is basically vegan except trace stuff, then there's butter too
# Snacks: can help on chips, but lots of brands, and doubles on milk, cheese, ingredient (does show granola too)
# Some Vegetables and Vegetable Products have milk b/c of processing, but fairly uncommon
# Nut and Seed Products - some oil mentions in there could throw it off, but otherwise decent; includes nut butters
# Finfish and Shellfish Products has raw, and so does Fruits, cut out? but could help w/ raw diets...
# Baked Products can include cheese, less of milk
# Sweets have egg, milk, honey, ice cream
# Cereal Grains and Pasta - some has egg
# Fast Foods has a lot of dairy, cheese, meats, pizza
# Meals, Entrees, and Sidedishes seems to have a lot of non-vegan stuff in general
# Restaurant Foods seems to have a lot of nonvegan and ethnic shit

array(['Dairy and Egg Products', 'Spices and Herbs', 'Baby Foods',
       'Fats and Oils', 'Poultry Products', 'Soups, Sauces, and Gravies',
       'Sausages and Luncheon Meats', 'Breakfast Cereals', 'Snacks',
       'Fruits and Fruit Juices', 'Pork Products',
       'Vegetables and Vegetable Products', 'Nut and Seed Products',
       'Beef Products', 'Beverages', 'Finfish and Shellfish Products',
       'Legumes and Legume Products', 'Lamb, Veal, and Game Products',
       'Baked Products', 'Sweets', 'Cereal Grains and Pasta', 'Fast Foods',
       'Meals, Entrees, and Sidedishes',
       'American Indian/Alaska Native Foods', 'Restaurant Foods'], dtype=object)

In [11]:
# Let's make a dictionary with the 25 unique food groups and their respective descriptions
deduped = []
for group, group_df in dads.groupby(['Food_Group', 'Description']):
    deduped.append({
      "Food_Group":  group[0],
      "Description": group[1]
            #imagine that I were able to grab the number of descriptions per food group here, it'd be redundant
            #here, but avoid the len check
    })

In [12]:
# This dictionary will give me the number of unique descriptions per food group
original_fg = {}
for classified in dads.Food_Group.unique():
    counter = 0
    for i in range(len(deduped)):
#   doesnt do anything
        if deduped[i]['Food_Group'] == classified:
            counter += 1
    original_fg[classified] = counter
# original_fg

In [13]:
number_dic = {
 'American Indian/Alaska Native Foods': range(0,165),
 'Baby Foods': range(165,498),
 'Baked Products': range(498, 1283),
 'Beef Products': range(1283, 2068),
 'Beverages': range(2068,2347),
 'Breakfast Cereals': range(2347,2719),
 'Cereal Grains and Pasta': range(2719,2903),
 'Dairy and Egg Products': range(2903,3157),
 'Fast Foods': range(3157,3552),
 'Fats and Oils': range(3552,3772),
 'Finfish and Shellfish Products': range(3772,4038),
 'Fruits and Fruit Juices': range(4038,4365),
 'Lamb, Veal, and Game Products': range(4365,4718),
 'Legumes and Legume Products': range(4718,5096),
 'Meals, Entrees, and Sidedishes': range(5096,5191),
 'Nut and Seed Products': range(5191,5320),
 'Pork Products': range(5320,5661),
 'Poultry Products': range(5661,5998),
 'Restaurant Foods': range(5998,6053),
 'Sausages and Luncheon Meats': range(6053,6287),
 'Snacks': range(6287,6458),
 'Soups, Sauces, and Gravies': range(6458,6973),
 'Spices and Herbs': range(6973,7034),
 'Sweets': range(7034,7378),
 'Vegetables and Vegetable Products': range(7378,8194)
}

In [14]:
fg_vallist_dic = {}
for classified in dads.Food_Group.unique():
    desc_list = []
    for item_i in number_dic[classified]:
        item_desc = deduped[item_i]['Description']
    
        desc_list.append(item_desc)
    
    fg_vallist_dic[classified] = desc_list
# fg_vallist_dic['Sweets'][343]

In [15]:
# Though that's okay, I'd prefer a dictionary with the number of words in each unique description per food group -
# this will help my proportion score down below surface even more relevant results
next_fg = {}
for classified in dads.Food_Group.unique():
    string_counter = 0
    for desc_string in usda[usda.Food_Group == classified].Description.unique():
        string_counter += len(desc_string.split())
    next_fg[classified] = string_counter
# next_fg

In [16]:
# Able to update as needed for experimentation
tmp_banned_cat = [
    'American Indian/Alaska Native Foods',
    'Baby Foods',
#     'Baked Products',
#     'Beef Products',
#     'Beverages',
#     'Breakfast Cereals',
#     'Cereal Grains and Pasta',
#     'Dairy and Egg Products',
    'Fast Foods',
#     'Fats and Oils',
#     'Finfish and Shellfish Products',
#     'Fruits and Fruit Juices',
#     'Lamb, Veal, and Game Products',
#     'Legumes and Legume Products',
    'Meals, Entrees, and Sidedishes',
#     'Nut and Seed Products',
#     'Pork Products',
#     'Poultry Products',
    'Restaurant Foods',
#     'Sausages and Luncheon Meats',
#     'Snacks',
#     'Soups, Sauces, and Gravies',
#     'Spices and Herbs',
#     'Sweets',
#     'Vegetables and Vegetable Products'
    ]

In [17]:
# Restrict how many non-zero counterized groups to return *more maleable function)
only_return_top_n = 100

In [18]:
def assign_groups(word_in_item):
    fg_dic = {}
    
    start_time = time.time()

    for grouped in dads.Food_Group.unique():
        if grouped not in tmp_banned_cat:
            counter_the = 0
            for item_i in range(original_fg[grouped]):
                search_this = fg_vallist_dic[grouped][item_i].lower().replace(',','').replace('/',' ').replace('-',' ').replace('_',' ').replace('.',' ').replace('(','').replace(')','').replace("'","").replace('&','')            
                if search_this.find(word_in_item.lower()) != -1:
                    counter_the += 1
                
            fg_dic[grouped] = float(counter_the) / next_fg[grouped]
#     Here is where I am improving my proportion with the ever-more accurate string counter dictionary    

    ordered_dict = sorted(fg_dic.items(), key=lambda x: x[1])
    dict_to_list = []
    for k,v in ordered_dict:
        if v != 0:
            new_list = [k,v]
            dict_to_list.append(new_list)
    dict_to_list = dict_to_list[::-1]
    max_groups = dict_to_list[:only_return_top_n]
    if max_groups != []:
        return max_groups
    
# , ("--- %s seconds ---" % (time.time() - start_time))

In [19]:
assign_groups('pork')

[['Pork Products', 0.09339596573639126],
 ['Sausages and Luncheon Meats', 0.07442596991290577],
 ['Legumes and Legume Products', 0.0021141649048625794],
 ['Snacks', 0.0017123287671232876],
 ['Soups, Sauces, and Gravies', 0.0015768725361366622],
 ['Baked Products', 0.0002166377816291161]]

In [20]:
# This function will assign food groups if given a feature from a particular item as an input

def groups_from_item(itemized):

    #clean item
    word_in_item = itemized.lower().replace(',','').replace('/',' ').replace('-',' ').replace('_',' ').replace('.',' ').replace('(','').replace(')','').replace("'","").replace('&','')
    
    if len(word_in_item.split()) == 1:
#     because there's just one word, we can feed the item directly into the search loop
        check_for_result = assign_groups(word_in_item)
        if check_for_result != None:
            all_food_groups = check_for_result
            all_food_groups = [i[0].replace(',','') for i in all_food_groups]
            all_food_groups = ', '.join(all_food_groups)
            return all_food_groups
        else:
            return None

    elif len(word_in_item.split()) >= 1:
        
        # because there's more than one word, I'll have to do an extra loop to hit each word, but making sure that
        # they get appended to the same row, and not to the following row
        
        each_words_group = []

        for word_per_item in word_in_item.split():
            mini_stop_words = ['and','with','by','for']            
            if word_per_item not in mini_stop_words:
                a_food_group = assign_groups(word_per_item)
                if a_food_group != None:
                    each_words_group += a_food_group

        if each_words_group != []:
            
            # This process considers all the groups that were labeled per word, and sums their proportionate scores
            all_food_groups = each_words_group
            unique_descs = [k for k in dict(all_food_groups).keys()]
            summed_groups = {}
            for key in unique_descs:
                tmp_list = []
                for i, level in enumerate(all_food_groups):
                    if key == all_food_groups[i][0]:
                        tmp_list.append(all_food_groups[i][1])
                summed_groups[key] = sum(tmp_list)

            ordered_dict = sorted(summed_groups.items(), key=itemgetter(1))
            dict_to_list = []
            for k,v in ordered_dict:
                if v != 0:
                    new_list = [k,v]
                    dict_to_list.append(new_list)
            dict_to_list = dict_to_list[::-1]
            max_groups = dict_to_list[:only_return_top_n]
            if max_groups != []:
                max_groups = [i[0].replace(',','') for i in max_groups]
                all_food_groups = ', '.join(max_groups)
                return all_food_groups
            
#     , ("--- %s seconds ---" % (time.time() - start_time))

# ^^ I think the best way to do this is to make fg columns for item, menu, sub_menu, etc. and then pick the
# group label with the highest label, or maybe the top 2/3 labels per row

In [21]:
hup = updated_vegan.sample(n=90)

In [22]:
start_time = time.time()
yup = hup.menu_item_name.map(lambda x: groups_from_item(x))
print ("--- %s seconds ---" % (time.time() - start_time))

--- 18.3622870445 seconds ---


In [24]:
# # previous script @ --- 20.9017841816 seconds ---, new script @ --- 20.2134339809 seconds ---
# start_time = time.time()
# # hup = updated_vegan.menu_item_name.sample(n=90)
# yup = hup.map(lambda x: groups_from_item(x))
# print ("--- %s seconds ---" % (time.time() - start_time))

In [25]:
updated_food.columns

Index([u'venue_name', u'venue_desc_list', u'vegan_venue_check',
       u'venue_menu_url', u'venue_rated', u'meta_menu_n', u'depth_menus_n',
       u'menu_item_name', u'menu_item_price', u'menu_item_desc',
       u'count_desc_list', u'count_item_name', u'count_item_desc'],
      dtype='object')

In [26]:
#issues - Mocha, Avocado, Slaw, Strawberry, Soup, Hamburger, Pastries, Bagel

In [27]:
word = 'mocha'
assign_groups(word), groups_from_item(word)

([['Beverages', 0.001034126163391934],
  ['Legumes and Legume Products', 0.00042283298097251583]],
 'Beverages, Legumes and Legume Products')

In [28]:
from ipywidgets import widgets
from IPython.display import display
search_food = widgets.Text(description="Search:", placeholder="Enter a food or ingredient!")
my_button = widgets.Button(description="This best be good...")

def on_submit(ui):
    print 'hey, i dont like it that rough'
#     print mytext.value
    
    to_predict = [(assign_groups(search_food.value))]
    print to_predict

my_button.on_click(on_submit)

display(search_food)
display(my_button)

In [29]:
list_run_thru = updated_vegan.menu_item_name
print len(list_run_thru), len(updated_food.menu_item_name)

1861 90047


In [30]:
# updated_food.menu_item_name

In [31]:
start_time = time.time()
print groups_from_item('Salty Fish and Chicken Fried Rice'), ("--- %s seconds ---" % (time.time() - start_time))

Finfish and Shellfish Products, Poultry Products, Cereal Grains and Pasta, Soups Sauces and Gravies, Sausages and Luncheon Meats, Snacks, Legumes and Legume Products, Breakfast Cereals, Fats and Oils, Vegetables and Vegetable Products, Pork Products, Sweets, Lamb Veal and Game Products, Baked Products, Fruits and Fruit Juices, Beverages, Dairy and Egg Products, Beef Products --- 0.381889104843 seconds ---


In [32]:
# # Here I've automated the food group tagging I would have done across each item, sub menu, and meta menu
# start_time = time.time()
# updated_food['item_fg'] = updated_food.menu_item_name.map(lambda x: groups_from_item(x))
# print("--- %s seconds ---" % (time.time() - start_time))
# # Completion time: --- 21613.890872 seconds ---

In [33]:
# # Here I've automated the food group tagging I would have done across each item, sub menu, and meta menu
# start_time = time.time()
# updated_food['item_fg_renewed'] = updated_food.menu_item_name.map(lambda x: groups_from_item(x))
# print("--- %s seconds ---" % (time.time() - start_time))
# # Completion time: --- 22567.7773201 seconds ---

In [34]:
# updated_food.item_fg.value_counts()

In [35]:
# # I also successfully ran this one:
# updated_food['depth_menu_fg'] = updated_food.depth_menus_n.map(lambda x: groups_from_item(x))

In [36]:
# updated_food['meta_menu_fg'] = updated_food.meta_menu_n.map(lambda x: groups_from_item(x))

In [37]:
# updated_food['item_desc_fg'] = updated_food.menu_item_desc.map(
#     lambda x: groups_from_item(x) if x != 'desc_not_available' else "no_groups_avail" )

In [ ]:
updated_food[['menu_item_name','menu_item_desc','item_fg','depth_menu_fg','meta_menu_fg','item_desc_fg']].head(5)

In [38]:
# updated_food.to_pickle('../../../../Accounts/Electo:Computad/\
# MacBook Pro/Software/Github/Old_repos/DSI-SF-4-anotherguitar/projects/Capstone Stuff/updated_food.pkl')

In [39]:
updated_food = pd.read_pickle('../../../../Accounts/Electo:Computad/\
MacBook Pro/Software/Github/Old_repos/DSI-SF-4-anotherguitar/projects/Capstone Stuff/updated_food.pkl')

In [40]:
updated_food.item_fg_renewed.shape

(90047,)

In [41]:
updated_food.depth_menu_fg.shape

(90047,)

In [42]:
# updated_food.item_fg.value_counts()

In [43]:
updated_food.columns

Index([u'venue_name', u'venue_desc_list', u'vegan_venue_check',
       u'venue_menu_url', u'venue_rated', u'meta_menu_n', u'depth_menus_n',
       u'menu_item_name', u'menu_item_price', u'menu_item_desc',
       u'count_desc_list', u'count_item_name', u'count_item_desc', u'item_fg',
       u'depth_menu_fg', u'item_fg_renewed'],
      dtype='object')

In [44]:
# Make a function that will put a one in the row if that group is represented, and 0 if not:
def check_group(ref_group, food_groups):
    
    if food_groups == None:
        return 0
    elif len(food_groups.split(',')) == 1:
        if ref_group.replace(',','') in food_groups:
            return 1
        else:
            return 0

    elif len(food_groups.split(',')) >= 1:
        for a_group in food_groups.split(','):
            if ref_group.replace(',','') in a_group:
                return 1
        return 0
    else:
        return 0

In [45]:
# updated_food = updated_food.drop([
#         'Dairy and Egg Products', 'Spices and Herbs', 'Fats and Oils',
#        'Poultry Products', 'Soups, Sauces, and Gravies',
#        'Sausages and Luncheon Meats', 'Breakfast Cereals', 'Snacks',
#        'Fruits and Fruit Juices', 'Pork Products',
#        'Vegetables and Vegetable Products', 'Nut and Seed Products',
#        'Beef Products', 'Beverages', 'Finfish and Shellfish Products',
#        'Legumes and Legume Products', 'Lamb, Veal, and Game Products',
#        'Baked Products', 'Sweets', 'Cereal Grains and Pasta',
#        'Meals, Entrees, and Sidedishes'
#     ], axis=1)
# updated_food.columns

In [47]:
for i in dads.Food_Group.unique():
    print 'next loop'
    if i.replace(',','') not in tmp_banned_cat:
        current_ref_group = i.replace(',','')
        updated_food[i] = updated_food.item_fg_renewed.map(lambda x: check_group(current_ref_group, x))

next loop
next loop
next loop
next loop
next loop
next loop
next loop
next loop
next loop
next loop
next loop
next loop
next loop
next loop
next loop
next loop
next loop
next loop
next loop
next loop
next loop
next loop
next loop
next loop
next loop


In [48]:
updated_food['Sausages and Luncheon Meats'].value_counts()

0    49008
1    41039
Name: Sausages and Luncheon Meats, dtype: int64

In [49]:
updated_food['Vegetables and Vegetable Products'].value_counts()
# Results from original script:
# 0    78210
# 1    11837
# Name: Vegetables and Vegetable Products, dtype: int64
# Results from improved (more comprehensive) script:
# 1    48323
# 0    41724
# Name: Vegetables and Vegetable Products, dtype: int64

1    48323
0    41724
Name: Vegetables and Vegetable Products, dtype: int64

In [50]:
updated_vegan = updated_food[updated_food.vegan_venue_check == 'vegan']

In [51]:
# updated_vegan.sample(90).T

In [52]:
# updated_food.sample(90).T

In [53]:
# updated_vegan.item_fg_renewed.value_counts()

In [54]:
# Below is extra data that I might later use to correlate nutrients with particular food groups,
# though I'm not sure if there will be enough time for me to complete this aspect of the project

In [55]:
usda.shape

(595359, 9)

In [56]:
tmp = usda[['Food_Group','Nutrient']]

In [57]:
tmp = tmp.groupby(['Food_Group','Nutrient']).agg(lambda x: 1.)
tmp.head()

Food_Group                           Nutrient
American Indian/Alaska Native Foods  10:0        1.0
                                     12:0        1.0
                                     14:0        1.0
                                     14:1        1.0
                                     15:0        1.0
dtype: float64

In [58]:
tmp = tmp.unstack(fill_value=0)

In [59]:
tmp.shape

(25, 145)

In [60]:
tmp = tmp.reset_index()

In [61]:
tmp['Carbohydrate, by difference']

0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
Name: Carbohydrate, by difference, dtype: float64

In [62]:
tmp.columns.unique()

array(['Food_Group', '10:0', '12:0', '13:0', '14:0', '14:1', '15:0',
       '15:1', '16:0', '16:1 c', '16:1 t', '16:1 undifferentiated', '17:0',
       '17:1', '18:0', '18:1 c', '18:1 t', '18:1 undifferentiated',
       '18:1-11t (18:1t n-7)', '18:2 CLAs', '18:2 i', '18:2 n-6 c,c',
       '18:2 t not further defined', '18:2 t,t', '18:2 undifferentiated',
       '18:3 n-3 c,c,c (ALA)', '18:3 n-6 c,c,c', '18:3 undifferentiated',
       '18:3i', '18:4', '20:0', '20:1', '20:2 n-6 c,c', '20:3 n-3',
       '20:3 n-6', '20:3 undifferentiated', '20:4 n-6',
       '20:4 undifferentiated', '20:5 n-3 (EPA)', '21:5', '22:0', '22:1 c',
       '22:1 t', '22:1 undifferentiated', '22:4', '22:5 n-3 (DPA)',
       '22:6 n-3 (DHA)', '24:0', '24:1 c', '4:0', '6:0', '8:0',
       'Adjusted Protein', 'Alanine', 'Alcohol, ethyl', 'Arginine', 'Ash',
       'Aspartic acid', 'Beta-sitosterol', 'Betaine', 'Caffeine',
       'Calcium, Ca', 'Campesterol', 'Carbohydrate, by difference',
       'Carotene, alpha', 'C

In [63]:
tmp['Sucrose'].head(3).T

0    1.0
1    1.0
2    1.0
Name: Sucrose, dtype: float64

In [64]:
tmp.head(10)

Nutrient,Food_Group,10:0,12:0,13:0,14:0,14:1,15:0,15:1,16:0,16:1 c,...,"Vitamin C, total ascorbic acid",Vitamin D,Vitamin D (D2 + D3),Vitamin D2 (ergocalciferol),Vitamin D3 (cholecalciferol),Vitamin E (alpha-tocopherol),"Vitamin E, added",Vitamin K (phylloquinone),Water,"Zinc, Zn"
0,American Indian/Alaska Native Foods,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1,Baby Foods,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,Baked Products,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,Beef Products,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,Beverages,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
5,Breakfast Cereals,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,Cereal Grains and Pasta,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
7,Dairy and Egg Products,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,Fast Foods,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
9,Fats and Oils,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
